In [1]:
from pyspark.sql import SparkSession
import getpass 
username=getpass.getuser()
spark=SparkSession. \
    builder. \
    config('spark.ui.port','0'). \
    config("spark.sql.warehouse.dir", f"/user/itv012740/warehouse"). \
    config('spark.shuffle.useOldFetchProtocol', 'true'). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

In [2]:
loans_def_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.option("inferSchema", True) \
.load("/user/itv012740/lendingclubproject/raw/loans_defaulters_csv")

In [3]:
loans_def_raw_df

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record
b59d80da191f5b573...,0.0,0.0,0.0,0.0,1.0,0.0,31.0,null
202d9f56ecb7c3bc9...,1.0,0.0,0.0,0.0,0.0,0.0,6.0,null
e5a140c0922b554b9...,0.0,0.0,0.0,0.0,0.0,0.0,47.0,null
e12aefc548f750777...,0.0,0.0,0.0,0.0,0.0,0.0,33.0,null
1b3a50d854fbbf97e...,1.0,0.0,0.0,0.0,0.0,0.0,21.0,null
1c4329e5f17697127...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
5026c86ad983175eb...,0.0,0.0,1.0,0.0,2.0,0.0,null,71.0
9847d8c1e9d0b2084...,1.0,0.0,2.0,0.0,0.0,0.0,6.0,63.0
8340dbe1adea41fb4...,0.0,0.0,0.0,0.0,0.0,0.0,36.0,null
d4de0de3ab7d79ad4...,0.0,0.0,0.0,0.0,0.0,0.0,35.0,null


In [4]:
loans_def_raw_df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- delinq_amnt: double (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- pub_rec_bankruptcies: double (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- total_rec_late_fee: string (nullable = true)
 |-- mths_since_last_delinq: string (nullable = true)
 |-- mths_since_last_record: string (nullable = true)



In [5]:
loans_def_raw_df.createOrReplaceTempView("loan_defaulters")

In [7]:
spark.sql("select distinct(delinq_2yrs) from loan_defaulters")

delinq_2yrs
1.0
271 monthly payme...
I bike to work on...
183xx
VISA and AMEX cre...
etc. and I feel t...
AZ
017xx
923xx
446xx


In [8]:
spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters group by delinq_2yrs order by total desc").show(40)

+------------------+-------+
|       delinq_2yrs|  total|
+------------------+-------+
|               0.0|1838878|
|               1.0| 281335|
|               2.0|  81285|
|               3.0|  29539|
|               4.0|  13179|
|               5.0|   6599|
|               6.0|   3717|
|               7.0|   2062|
|               8.0|   1223|
|               9.0|    818|
|              10.0|    556|
|              11.0|    363|
|              12.0|    264|
|              13.0|    165|
|              14.0|    120|
|              15.0|     87|
|              null|     63|
|              16.0|     55|
|              17.0|     30|
|              18.0|     30|
|              19.0|     23|
|              20.0|     17|
|              21.0|     12|
|                CA|      8|
|                TX|      6|
|                IL|      5|
|              22.0|      5|
|    small_business|      5|
|debt_consolidation|      5|
|              24.0|      4|
|                FL|      4|
|             

In [9]:
loan_defaulters_schema = "member_id string, delinq_2yrs float, delinq_amnt float, pub_rec float, pub_rec_bankruptcies float,inq_last_6mths float, total_rec_late_fee float, mths_since_last_delinq float, mths_since_last_record float"

In [10]:
loans_def_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.schema(loan_defaulters_schema) \
.load("/user/itv012740/lendingclubproject/raw/loans_defaulters_csv")

In [11]:
loans_def_raw_df.createOrReplaceTempView("loan_defaulters")

In [12]:
spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters group by delinq_2yrs order by total desc").show(40)

+-----------+-------+
|delinq_2yrs|  total|
+-----------+-------+
|        0.0|1838878|
|        1.0| 281335|
|        2.0|  81285|
|        3.0|  29539|
|        4.0|  13179|
|        5.0|   6599|
|        6.0|   3717|
|        7.0|   2062|
|        8.0|   1223|
|        9.0|    818|
|       10.0|    556|
|       11.0|    363|
|       12.0|    264|
|       null|    261|
|       13.0|    165|
|       14.0|    120|
|       15.0|     87|
|       16.0|     55|
|       18.0|     30|
|       17.0|     30|
|       19.0|     23|
|       20.0|     17|
|       21.0|     12|
|       22.0|      5|
|       24.0|      4|
|       26.0|      3|
|       23.0|      2|
|       25.0|      2|
|       3.44|      2|
|       30.0|      2|
|       29.0|      2|
|       3.45|      1|
|      13.76|      1|
|      21.72|      1|
|       58.0|      1|
|      17.17|      1|
|       6.52|      1|
|       5.52|      1|
|      22.95|      1|
|      22.62|      1|
+-----------+-------+
only showing top 40 rows



In [13]:
from pyspark.sql.functions import col

In [14]:
loans_def_processed_df = loans_def_raw_df.withColumn("delinq_2yrs", col("delinq_2yrs").cast("integer")).fillna(0, subset = ["delinq_2yrs"])

In [15]:
loans_def_processed_df.createOrReplaceTempView("loan_defaulters")

In [16]:
spark.sql("select count(*) from loan_defaulters where delinq_2yrs is null")

count(1)
0


In [17]:
spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters group by delinq_2yrs order by total desc").show(40)

+-----------+-------+
|delinq_2yrs|  total|
+-----------+-------+
|          0|1839141|
|          1| 281337|
|          2|  81285|
|          3|  29545|
|          4|  13180|
|          5|   6601|
|          6|   3719|
|          7|   2063|
|          8|   1226|
|          9|    821|
|         10|    558|
|         11|    363|
|         12|    266|
|         13|    167|
|         14|    123|
|         15|     90|
|         16|     56|
|         17|     33|
|         18|     32|
|         19|     24|
|         20|     19|
|         21|     16|
|         22|      7|
|         24|      6|
|         23|      5|
|         26|      4|
|         29|      2|
|         25|      2|
|         30|      2|
|         28|      1|
|         27|      1|
|         32|      1|
|         35|      1|
|         39|      1|
|         58|      1|
|         42|      1|
|         36|      1|
+-----------+-------+



In [18]:
loans_def_delinq_df = spark.sql("select member_id,delinq_2yrs, delinq_amnt, int(mths_since_last_delinq) from loan_defaulters where delinq_2yrs > 0 or mths_since_last_delinq > 0")

In [19]:
loans_def_delinq_df

member_id,delinq_2yrs,delinq_amnt,mths_since_last_delinq
b59d80da191f5b573...,0,0.0,31
202d9f56ecb7c3bc9...,1,0.0,6
e5a140c0922b554b9...,0,0.0,47
e12aefc548f750777...,0,0.0,33
1b3a50d854fbbf97e...,1,0.0,21
9847d8c1e9d0b2084...,1,0.0,6
8340dbe1adea41fb4...,0,0.0,36
d4de0de3ab7d79ad4...,0,0.0,35
1d4e1ef4353b73c00...,0,0.0,30
6f196952e71277fd4...,4,0.0,5


In [20]:
loans_def_delinq_df.count()

1106163

In [21]:
loans_def_records_enq_df = spark.sql("select member_id from loan_defaulters where pub_rec > 0.0 or pub_rec_bankruptcies > 0.0 or inq_last_6mths > 0.0")

In [22]:
loans_def_records_enq_df

member_id
b59d80da191f5b573...
5026c86ad983175eb...
9847d8c1e9d0b2084...
9dd72636b1b4045b4...
1d4e1ef4353b73c00...
9c617cbc6e3e3d6a1...
45eb04dd75400a942...
08d233c81f9e50726...
8ff4d0ed17a1cab92...
aec13cfd611b2cfea...


In [23]:
loans_def_records_enq_df.count()

1070125

In [24]:
loans_def_delinq_df.write \
.option("header", True) \
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv012740/lendingclubproject/raw/cleaned/loans_defaulters_deling_csv") \
.save()

In [25]:
loans_def_delinq_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv012740/lendingclubproject/raw/cleaned/loans_defaulters_deling_parquet") \
.save()

In [26]:
loans_def_records_enq_df.write \
.option("header", True) \
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv012740/lendingclubproject/raw/cleaned/loans_defaulters_records_enq_csv") \
.save()

In [27]:
loans_def_records_enq_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv012740/lendingclubproject/raw/cleaned/loans_defaulters_records_enq_parquet") \
.save()

In [28]:
loans_def_p_pub_rec_df = loans_def_processed_df.withColumn("pub_rec", col("pub_rec").cast("integer")).fillna(0, subset = ["pub_rec"])

In [29]:
loans_def_p_pub_rec_bankruptcies_df = loans_def_p_pub_rec_df.withColumn("pub_rec_bankruptcies", col("pub_rec_bankruptcies").cast("integer")).fillna(0, subset = ["pub_rec_bankruptcies"])

In [30]:
loans_def_p_inq_last_6mths_df = loans_def_p_pub_rec_bankruptcies_df.withColumn("inq_last_6mths", col("inq_last_6mths").cast("integer")).fillna(0, subset = ["inq_last_6mths"])

In [31]:
loans_def_p_inq_last_6mths_df.createOrReplaceTempView("loan_defaulters")

In [32]:
loans_def_detail_records_enq_df = spark.sql("select member_id, pub_rec, pub_rec_bankruptcies, inq_last_6mths from loan_defaulters")

In [33]:
loans_def_detail_records_enq_df

member_id,pub_rec,pub_rec_bankruptcies,inq_last_6mths
b59d80da191f5b573...,0,0,1
202d9f56ecb7c3bc9...,0,0,0
e5a140c0922b554b9...,0,0,0
e12aefc548f750777...,0,0,0
1b3a50d854fbbf97e...,0,0,0
1c4329e5f17697127...,0,0,0
5026c86ad983175eb...,1,0,2
9847d8c1e9d0b2084...,2,0,0
8340dbe1adea41fb4...,0,0,0
d4de0de3ab7d79ad4...,0,0,0


In [34]:
loans_def_detail_records_enq_df.write \
.option("header", True) \
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv012740/lendingclubproject/raw/cleaned/loans_def_detail_records_enq_df_csv") \
.save()

In [35]:
loans_def_detail_records_enq_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv012740/lendingclubproject/raw/cleaned/loans_def_detail_records_enq_df_parquet") \
.save()